In [1]:
import gurobipy as gp
import pandas as pd
from code_map import data_handling, utils, new_meters, final_markets
import numpy as np
import calendar 
from datetime import datetime
import pytz
import openpyxl
import os
import pickle


In [2]:
#return L, M, F, H, freq_data, power_meter_dict, consumption_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m, dominant_directions, Ir_hlm, Ia_hlm, Va_hm, compatible_list
L, M, F, H, freq_data, power_meter_dict, consumption_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m, dominant_directions, Ir_hlm, Ia_hlm, Va_hm, compatible_list = data_handling.load_collections("./one_day_collections.pkl")

In [3]:
print(f"Amount of markets : {len(M)}")
print(f"Amount of meters : {len(L)}")
print(f"Amount of meters with direction up or both : {len(L_u)}")
print(f"Amount of meters with direction down or both : {len(L_d)}")
print(f"Amount of hours : {len(H)}")

Amount of markets : 62
Amount of meters : 2189
Amount of meters with direction up or both : 2036
Amount of meters with direction down or both : 2051
Amount of hours : 24


In [4]:
one_week_model, x, y, w, d  = utils.run_optimization_model_no_numpy(L, M, H, F, Ir_hlm, Ia_hlm, Va_hm, Vp_h_m, Vm_m, compatible_list, "weekly_test_model.log", "one_week_model")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-13
Set parameter LogFile to value "weekly_test_model.log"
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 23.1.0 23B92)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6573432 rows, 3260208 columns and 20327160 nonzeros
Model fingerprint: 0x35c46531
Model has 133324 quadratic objective terms
Model has 1728 quadratic constraints
Variable types: 0 continuous, 3260208 integer (3260208 binary)
Coefficient statistics:
  Matrix range     [1e-06, 2e+03]
  QMatrix range    [1e-06, 8e-01]
  Objective range  [5e-06, 2e+05]
  QObjective range [1e-06, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [2e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 6518237 rows and 1855190 columns (presolve time = 5s) ...
Presolve removed 6518237 rows and 18

In [5]:
utils.test_solution_validity(x, y, w, Va_hm, L, M, H, dominant_directions= dominant_directions, F = F)

'Solution is valid'

In [6]:
new_x_values = {(h, l, m): one_week_model.getVarByName(f"x_{h}_{l}_{m}").X for h in range(len(H)) for l in range(len(L)) for m in range(len(M))}

In [8]:
utils.compare_current_and_new_solution(current_x_pkl_file= "current_x_values_for_day.pkl", new_x_values= new_x_values, L= L, M= M, H= H, dominant_directions= dominant_directions)

{}

In [ ]:

# Save these values
with open('current_x_values_for_week.pkl', 'wb') as f:
    pickle.dump(new_x_values, f)
                